# FastPitch Adapter Finetuning

This notebook is designed to provide a guide on how to run FastPitch Adapter Finetuning Pipeline. It contains the following sections:
1. **Fine-tune FastPitch on adaptation data**: fine-tune pre-trained multi-speaker FastPitch for a new speaker
* Dataset Preparation: download dataset and extract manifest files. (duration more than 15 mins)
* Preprocessing: add absolute audio paths and normalized texts in manifest, calculate pitch stats, extract speaker embedding from pre-trained speaker-verification (SV) model.
* Training: fine-tune frozen multispeaker FastPitch with trainable adapters and weighted looked-up speaker embedding.
2. **Fine-tune HiFiGAN on adaptation data**: fine-tune a vocoder for the fine-tuned multi-speaker FastPitch
* Dataset Preparation: extract mel-spectrograms from fine-tuned FastPitch.
* Training: fine-tune HiFiGAN with fine-tuned adaptation data.
3. **Inference**: generate speech from adpated FastPitch
* Load Model: load pre-trained multi-speaker FastPitch with fine-tuned adapters and weighted looked-up speaker embedding.
* Output Audio: generate audio files.

# License

> Copyright 2023 NVIDIA. All Rights Reserved.
> 
> Licensed under the Apache License, Version 2.0 (the "License");
> you may not use this file except in compliance with the License.
> You may obtain a copy of the License at
> 
>     http://www.apache.org/licenses/LICENSE-2.0
> 
> Unless required by applicable law or agreed to in writing, software
> distributed under the License is distributed on an "AS IS" BASIS,
> WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
> See the License for the specific language governing permissions and
> limitations under the License.

In [ ]:
"""
You can either run this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.
Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies# .
"""
BRANCH = 'main'
# # If you're using Colab and not running locally, uncomment and run this cell.
# !apt-get install sox libsndfile1 ffmpeg
# !pip install wget unidecode pynini==2.1.4 scipy==1.7.3
# !python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

In [ ]:
sample_rate = 44100
# Store all python script
codedir = 'NeMoTTS' 
# Store all manifest and audios
datadir = 'NeMoTTS_dataset'
# Store all related text-normalized files
normdir = 'NeMoTTS_normalize_files'
# Store all supplementary files
suppdir = "NeMoTTS_sup_data"
# Store all config files
confdir = "NeMoTTS_conf"
# Store all training logs
logsdir = "NeMoTTS_logs"
# Store all mel-spectrograms for vocoder training
melsdir = "NeMoTTS_mels"
# Store all generated audios
gensdir = "NeMoTTS_gens"

In [ ]:
YOUR_PRETRAINED_FASTPITCH_CHECKPOINT = ""
YOUR_FINETUNED_HIFIGAN_ON_MULTISPEAKER_CHECKPOINT = ""

In [ ]:
!wandb login #PASTE_WANDB_APIKEY_HERE

In [ ]:
import os
import json
import nemo
import torch
import numpy as np

from pathlib import Path
from tqdm import tqdm

# 1. Fine-tune FastPitch on adaptation data

## a. Data Preparation
For our tutorial, we use small part of VCTK dataset with a new target speaker (p267). Usually, the audios should have total duration more than 15 mintues.

In [ ]:
!cd {datadir} && wget https://vctk-subset.s3.amazonaws.com/vctk_subset.tar.gz && tar zxf vctk_subset.tar.gz

In [ ]:
manidir = f"{datadir}/vctk_subset"
!ls {manidir}

For simplicity, we use original dev set as training set and original test set as validation set.

In [ ]:
train_manifest = os.path.abspath(os.path.join(manidir, 'train.json'))
valid_manifest = os.path.abspath(os.path.join(manidir, 'dev.json'))

## b. Preprocessing

### Add absolute file path in manifest

In [ ]:
def json_reader(filename):
    lines = []
    with open(filename) as f:
        for line in f: lines.append(json.loads(line))
    return lines

def json_writer(manifest, filename):
    with open(filename, 'w') as fout:
        for m in manifest: fout.write(json.dumps(m) + '\n') 

In [ ]:
train_datas = json_reader(train_manifest)
for m in train_datas: m['audio_filepath'] = os.path.abspath(os.path.join(manidir, m['audio_filepath']))
json_writer(train_datas, train_manifest)

In [ ]:
valid_datas = json_reader(valid_manifest)
for m in valid_datas: m['audio_filepath'] = os.path.abspath(os.path.join(manidir, m['audio_filepath']))
json_writer(valid_datas, valid_manifest)

### Calibrate speaker id to start from 0

In [ ]:
train_datas = json_reader(train_manifest)
for m in train_datas: m['old_speaker'], m['speaker'] = m['speaker'], 0
json_writer(train_datas, train_manifest)

valid_datas = json_reader(valid_manifest)
for m in valid_datas: m['old_speaker'], m['speaker'] = m['speaker'], 0
json_writer(valid_datas, valid_manifest)

### Extract speaker embedding from pre-trained speaker-verification model 

In [ ]:
!(cd {codedir} && wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/scripts/dataset_processing/tts/add_pretrained_speaker_embedding.py)

In [ ]:
!(cd {codedir} && python add_pretrained_speaker_embedding.py --manifest-path={train_manifest} --feature-dir={os.path.abspath(suppdir)})

In [ ]:
!(cd {codedir} && python add_pretrained_speaker_embedding.py --manifest-path={valid_manifest} --feature-dir={os.path.abspath(suppdir)})

### Add normalized text in manifest

In [ ]:
cd {codedir} && wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/scripts/dataset_processing/add_normalized_text.py

In [ ]:
!(cd {codedir} && python add_normalized_text.py --src {train_manifest} --dst {train_manifest}) 

In [ ]:
!(cd {codedir} && python add_normalized_text.py --src {valid_manifest} --dst {valid_manifest}) 

### Calculate Pitch Stats

In [ ]:
import librosa
from nemo.collections.asr.parts.preprocessing.features import WaveformFeaturizer
from nemo.collections.tts.parts.utils.tts_dataset_utils import get_base_dir

In [ ]:
def get_pitch(sample):    
    rel_audio_path = Path(sample["audio_filepath"]).relative_to(base_data_dir).with_suffix("")
    rel_audio_path_as_text_id = str(rel_audio_path).replace("/", "_")
    pitch_filepath = os.path.join(pitch_dir, f"{rel_audio_path_as_text_id}.pt")
    
    if os.path.exists(pitch_filepath):
        pitch = torch.load(pitch_filepath).numpy()

    else:
        features = wave_model.process(
            sample["audio_filepath"]
        )
        voiced_tuple = librosa.pyin(
            features.numpy(),
            fmin=librosa.note_to_hz('C2'),
            fmax=librosa.note_to_hz('C7'),
            frame_length=2048,
            sr=44100,
            fill_na=0.0,
        )
        pitch = voiced_tuple[0]
        torch.save(torch.from_numpy(pitch).float(), pitch_filepath)
    
    return pitch

In [ ]:
wave_model = WaveformFeaturizer(sample_rate=sample_rate)
pitch_dir = os.path.join(suppdir, 'pitch')
os.makedirs(pitch_dir, exist_ok=True)

train_pitchs = []
train_datas = json_reader(train_manifest)
base_data_dir = get_base_dir([item["audio_filepath"] for item in train_datas])
for m in tqdm(train_datas): train_pitchs.append(get_pitch(m))
    
valid_datas = json_reader(valid_manifest)
base_data_dir = get_base_dir([item["audio_filepath"] for item in valid_datas])
for m in tqdm(valid_datas): get_pitch(m)

train_pitchs = np.concatenate(train_pitchs)
pitch_mean = float(np.mean(train_pitchs))
pitch_std = float(np.std(train_pitchs))

with open(os.path.join(manidir, 'pitch_stats.json'), 'w') as f:
    json.dump({'pitch':[pitch_mean, pitch_std]}, f)

## c. Training

In [ ]:
!cd {codedir} \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/tts/fastpitch_finetune_adapters.py

In [ ]:
# Normally 100 epochs

!(python {codedir}/fastpitch_finetune_adapters.py \
--config-path={os.path.abspath(confdir)} \
--config-name=fastpitch_speaker_adaptation.yaml \
sample_rate=44100 \
train_dataset={train_manifest} \
validation_datasets={valid_manifest} \
sup_data_types="['align_prior_matrix', 'pitch', 'speaker_id', 'reference_audio']" \
sup_data_path={suppdir} \
+init_from_ptl_ckpt={YOUR_PRETRAINED_FASTPITCH_CHECKPOINT} \
pitch_mean={pitch_mean} \
pitch_std={pitch_std} \
model.n_speakers=10 \
model.speaker_emb_condition_prosody=True \
model.speaker_emb_condition_decoder=True \
model.speaker_emb_condition_aligner=True \
model.speaker_emb_condition_layernm=True \
model.speaker_embedding_dim=192 \
model.adapter.add_weight_speaker=True \
+model.text_tokenizer.add_blank_at=True \
phoneme_dict_path={normdir}/cmudict-0.7b_nv22.10 \
heteronyms_path={normdir}/heteronyms-052722 \
model.train_ds.dataloader_params.batch_size=8 \
model.validation_ds.dataloader_params.batch_size=8 \
model.train_ds.dataloader_params.num_workers=8 \
model.validation_ds.dataloader_params.num_workers=8 \
model.optim.lr=2e-4 \
~model.optim.sched \
model.optim.name=adam \
model.optim.weight_decay=0.0 \
exp_manager.exp_dir={logsdir} \
+exp_manager.create_wandb_logger=True \
+exp_manager.wandb_logger_kwargs.name="tutorial-FastPitch-finetune-adaptation" \
+exp_manager.wandb_logger_kwargs.project="NeMo" \
+exp_manager.checkpoint_callback_params.save_top_k=-1 \
trainer.max_epochs=10 \
trainer.check_val_every_n_epoch=10 \
trainer.log_every_n_steps=1 \
trainer.devices=1 \
trainer.strategy=ddp \
trainer.precision=32 \
)

# 4. Fine-tune HiFiGAN on adaptation data

## a. Dataset Preparation

In [ ]:
from nemo.collections.tts.parts.utils.tts_dataset_utils import BetaBinomialInterpolator
from nemo.collections.tts.models import FastPitchModel
from nemo.collections.tts.modules.submodules import WeightedSpeakerEmbedding
from collections import defaultdict
import random

In [ ]:
def gen_spectrogram(index, manifest, speaker_to_index, mel_dir, pitch_dir, base_data_dir):
    
    record = manifest[index]
    audio_file = record["audio_filepath"]
    
    if '.wav' in audio_file:
        save_path = os.path.abspath(os.path.join(mel_dir, audio_file.split("/")[-1].replace(".wav", ".npy")))
    
    if '.flac' in audio_file:
        save_path = os.path.abspath(os.path.join(mel_dir, audio_file.split("/")[-1].replace(".flac", ".npy")))
    
    if os.path.exists(save_path):
        return save_path
    
    if "normalized_text" in record:
        text = spec_model.parse(record["normalized_text"], normalize=False)
    else:
        text = spec_model.parse(record['text'])
        
    text_len = torch.tensor(text.shape[-1], dtype=torch.long, device=spec_model.device).unsqueeze(0)
    
    audio = wave_model.process(audio_file).unsqueeze(0).to(device=spec_model.device)
    audio_len = torch.tensor(audio.shape[1]).long().unsqueeze(0).to(device=spec_model.device)
    spect, spect_len = spec_model.preprocessor(input_signal=audio, length=audio_len) 
    
    rel_audio_path = Path(audio_file).relative_to(base_data_dir).with_suffix("")
    rel_audio_path_as_text_id = str(rel_audio_path).replace("/", "_")
    pitch_filepath = os.path.join(pitch_dir, f"{rel_audio_path_as_text_id}.pt")
    pitch = torch.load(pitch_filepath)
    pitch -= pitch_mean
    pitch[pitch == -pitch_mean] = 0.0
    pitch /= pitch_std
    pitch = pitch.unsqueeze(0).to(device=spec_model.device)
    attn_prior = torch.from_numpy(beta_binomial_interpolator(spect_len.item(), text_len.item())).unsqueeze(0).to(spec_model.device)
    
    speaker = torch.tensor([record['speaker']]).to(spec_model.device)
    
    reference_pool = speaker_to_index[record["speaker"]] - set([index]) if len(speaker_to_index[record["speaker"]]) > 1 else speaker_to_index[record["speaker"]]
    reference_sample = manifest[random.sample(reference_pool, 1)[0]]
    reference_audio = wave_model.process(reference_sample["audio_filepath"]).unsqueeze(0).to(device=spec_model.device)
    reference_audio_length = torch.tensor(reference_audio.shape[1]).long().unsqueeze(0).to(device=spec_model.device)
    reference_spec, reference_spec_len = spec_model.preprocessor(input_signal=reference_audio, length=reference_audio_length)  
    
        
    with torch.no_grad():
        spectrogram = spec_model.forward(
          text=text, 
          input_lens=text_len,
          pitch=pitch,
          spec=spect, 
          mel_lens=spect_len, 
          attn_prior=attn_prior,
          speaker=speaker,
          reference_spec=reference_spec,
          reference_spec_lens=reference_spec_len,
        )[0]
    
    spec = spectrogram[0].to('cpu').numpy()
    np.save(save_path, spec)
    return save_path

In [ ]:
YOUR_FINETUNED_FASTPITCH_CHECKPOINT_FOLDER = "/home/chsieh/NeMo/NeMoTTS_logs/FastPitch/2023-04-10_21-09-10"

# Pretrained FastPitch Weights
spec_model = FastPitchModel.load_from_checkpoint(YOUR_PRETRAINED_FASTPITCH_CHECKPOINT, strict=False)

# Load Adapter Weights
spec_model.load_adapters(f'{YOUR_FINETUNED_FASTPITCH_CHECKPOINT_FOLDER}/checkpoints/adapters.pt')
spec_model.freeze()
spec_model.unfreeze_enabled_adapters()

# Load Weighted Speaker Embedding Weights
wemb = WeightedSpeakerEmbedding(pretrained_embedding=spec_model.fastpitch.speaker_emb)
wemb.embedding_weight.data = torch.load(f'{YOUR_FINETUNED_FASTPITCH_CHECKPOINT_FOLDER}/checkpoints/wemb.pt')
wemb.embedding_weight.requires_grad = False
spec_model.fastpitch.speaker_emb = wemb
spec_model.eval().cuda()

beta_binomial_interpolator = BetaBinomialInterpolator()

In [ ]:
os.makedirs(melsdir, exist_ok=True)

# Train
train_datas = json_reader(train_manifest)
base_data_dir = get_base_dir([item["audio_filepath"] for item in train_datas])
speaker_to_index = defaultdict(list)
for i, d in enumerate(train_datas): speaker_to_index[d.get('speaker', None)].append(i)
speaker_to_index = {k: set(v) for k, v in speaker_to_index.items()}

for i, record in enumerate(tqdm(train_datas)):
    record["mel_filepath"] =  gen_spectrogram(i, train_datas, speaker_to_index, melsdir, pitch_dir, base_data_dir)

json_writer(train_datas, train_manifest)


# Valid
valid_datas = json_reader(valid_manifest)
base_data_dir = get_base_dir([item["audio_filepath"] for item in valid_datas])
speaker_to_index = defaultdict(list)
for i, d in enumerate(valid_datas): speaker_to_index[d.get('speaker', None)].append(i)
speaker_to_index = {k: set(v) for k, v in speaker_to_index.items()}

for i, record in enumerate(tqdm(valid_datas)):
    record["mel_filepath"] =  gen_spectrogram(i, valid_datas, speaker_to_index, melsdir, pitch_dir, base_data_dir)

json_writer(valid_datas, valid_manifest)

## b. Training

In [ ]:
!cd {confdir} \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/tts/conf/hifigan/hifigan_44100.yaml
!cd {confdir} \
&& mkdir -p model/train_ds && cd model/train_ds && wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/examples/tts/conf/hifigan/model/train_ds/train_ds_finetune.yaml 
!cd {confdir} \
&& mkdir -p model/validation_ds && cd model/validation_ds && wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/examples/tts/conf/hifigan/model/validation_ds/val_ds_finetune.yaml
!cd {confdir} \
&& mkdir -p model/generator && cd model/generator && wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/examples/tts/conf/hifigan/model/generator/v1_44100.yaml
!cd {codedir} \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/tts/hifigan_finetune.py

In [ ]:
# Normally 500 epochs 
!(python {codedir}/hifigan_finetune.py \
--config-path={os.path.abspath(confdir)} \
--config-name=hifigan_44100.yaml \
train_dataset={train_manifest} \
validation_datasets={valid_manifest} \
+init_from_ptl_ckpt={YOUR_FINETUNED_HIFIGAN_ON_MULTISPEAKER_CHECKPOINT} \
model.train_ds.dataloader_params.batch_size=32 \
model.optim.lr=0.0001 \
+trainer.max_epochs=20 \
trainer.check_val_every_n_epoch=20 \
model/train_ds=train_ds_finetune \
model/validation_ds=val_ds_finetune \
trainer.devices=1 \
trainer.strategy='ddp' \
trainer.precision=16 \
exp_manager.exp_dir={logsdir} \
exp_manager.create_wandb_logger=True \
exp_manager.wandb_logger_kwargs.name="tutorial-HiFiGAN" \
exp_manager.wandb_logger_kwargs.project="NeMo" \
)

# 5. Inference

In [ ]:
from nemo.collections.tts.models import HifiGanModel
import IPython.display as ipd

## a. Load Model

In [ ]:
wave_model = WaveformFeaturizer(sample_rate=sample_rate)

In [ ]:
# FastPitch
spec_model = FastPitchModel.load_from_checkpoint(YOUR_PRETRAINED_FASTPITCH_CHECKPOINT, strict=False)
spec_model.load_adapters(f'{YOUR_FINETUNED_FASTPITCH_CHECKPOINT_FOLDER}/checkpoints/adapters.pt')
spec_model.freeze()
spec_model.unfreeze_enabled_adapters()
wemb = WeightedSpeakerEmbedding(pretrained_embedding=spec_model.fastpitch.speaker_emb)
wemb.embedding_weight.data = torch.load(f'{YOUR_FINETUNED_FASTPITCH_CHECKPOINT_FOLDER}/checkpoints/wemb.pt')
wemb.embedding_weight.requires_grad = False
spec_model.fastpitch.speaker_emb = wemb
spec_model.eval().cuda()

In [ ]:
# HiFiGAN
YOUR_FINETUNED_HIFIGAN_ON_ADAPTATION_CHECKPOINT = ""
vocoder_model = HifiGanModel.load_from_checkpoint(checkpoint_path=YOUR_FINETUNED_HIFIGAN_ON_ADAPTATION_CHECKPOINT)
vocoder_model.eval().cuda()

## b. Output Audio

In [ ]:
def gt_spectrogram(audio_path, wave_model, spec_gen_model):
    features = wave_model.process(audio_path, trim=False)
    audio, audio_length = features, torch.tensor(features.shape[0]).long()
    audio = audio.unsqueeze(0).to(device=spec_gen_model.device)
    audio_length = audio_length.unsqueeze(0).to(device=spec_gen_model.device)
    with torch.no_grad():
        spectrogram, spec_len = spec_gen_model.preprocessor(input_signal=audio, length=audio_length)
    return spectrogram, spec_len

def gen_spectrogram(text, spec_gen_model, speaker, reference_spec, reference_spec_lens):
    parsed = spec_gen_model.parse(text)
    speaker = torch.tensor([speaker]).long().to(device=spec_gen_model.device)
    with torch.no_grad():    
        spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed, 
                                                          speaker=speaker, 
                                                          reference_spec=reference_spec, 
                                                          reference_spec_lens=reference_spec_lens)

    return spectrogram
  
def synth_audio(vocoder_model, spectrogram):    
    with torch.no_grad():  
        audio = vocoder_model.convert_spectrogram_to_audio(spec=spectrogram)
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return audio

In [ ]:
os.makedirs(gensdir, exist_ok=True)

# Reference Audio
with open(train_manifest, "r") as f:
    for i, line in enumerate(f):
        reference_record = json.loads(line)
        break
        
# Validatation Audio
num_val = 10
val_records = []
with open(valid_manifest, "r") as f:
    for i, line in enumerate(f):
        val_records.append(json.loads(line))
        if len(val_records) >= num_val:
            break

In [ ]:
with open(os.path.join(gensdir, f'manifest.json'), 'w') as f_manifest: 
    for i, val_record in enumerate(val_records):
        f_manifest.write(json.dumps(val_record) + '\n')
        
        audio = ipd.Audio(val_record['audio_filepath'], rate=sample_rate)
        with open(os.path.join(gensdir, f'{i}-GT.wav'), 'wb') as f: f.write(audio.data)

        reference_spec, reference_spec_lens = gt_spectrogram(reference_record['audio_filepath'], wave_model, spec_model)
        reference_spec = reference_spec.to(spec_model.device)
        spec_pred = gen_spectrogram(val_record['text'], spec_model,
                                    speaker=val_record['speaker'], 
                                    reference_spec=reference_spec, reference_spec_lens=reference_spec_lens)

        audio = synth_audio(vocoder_model, spec_pred)
        audio = ipd.Audio(audio, rate=sample_rate)
        with open(os.path.join(gensdir, f'{i}-Gen.wav'), 'wb') as f: f.write(audio.data)